
#### Author: Christopher Agard

In [1]:
import pandas as pd
import numpy as np
import os,glob

pd.options.display.max_columns = 50

# Setting source and output folders

In [2]:
tu={'worksource': "C:/Users/tuh27554/Documents/Grading/2019/FALL EX1/Input Data",
    'workoutput': "C:/Users/tuh27554/Documents/Grading/2019/FALL EX1/Output Data"}
gandolf={'worksource': "C:/Users/craga/Google Drive/TU 2019/Fall/Exam 1/Input data",
    'workoutput': "C:/Users/craga/Google Drive/TU 2019/Fall/Exam 1/Output Data"}
source = tu

# Getting and Cleaning Exam Data

In [3]:
os.chdir(source['worksource'])
files = glob.glob('*.csv')
files

['1111 exam 1 S2019 keys.csv',
 'Cross-Walk 2019 Fall to 2018 Fall Exams.csv',
 'Fall 2018v1.csv',
 'Fall 2019v1.csv']

Now we will read in the summaries for 2018 EX1 and 2019 EX1 and merge them as a separate comparison file.

In [14]:
df19_1.columns

Index(['F19E1', 'A', 'B', 'C', 'D', ' ', '0', 'F18E1'], dtype='object')

In [13]:
df18_1 = pd.read_csv('Fall 2018v1.csv').drop(columns = 'Unnamed: 0').rename(columns = {'A':'A_18','B':'B_18','C':'C_18',
                                                                                     'D':'D_18',' ':' _18'})
df18_1.head()

,F18E1,A_18,B_18,C_18,D_18,_18
0,1,8.0,1,90,1.0,NaN
1,2,93.0,5,2,NaN,NaN
2,3,1.0,26,71,1.0,NaN
3,4,10.0,3,83,4.0,NaN
4,5,1.0,83,7,9.0,NaN


In [15]:
df19_1 = pd.read_csv('Fall 2019v1.csv').drop(columns = 'Unnamed: 0').rename(columns = {'A':'A_19','B':'B_19','C':'C_19',
                                                                                     'D':'D_19',' ':' _19','0':'0_19'})
df19_1.head()

,F19E1,A_19,B_19,C_19,D_19,_19,0_19,F18E1
0,1,8.0,2,89.0,1.0,NaN,NaN,1
1,2,97.0,1,2.0,NaN,NaN,NaN,2
2,3,NaN,79,20.0,1.0,NaN,NaN,4
3,4,1.0,86,6.0,7.0,NaN,NaN,5
4,5,5.0,3,87.0,5.0,NaN,NaN,3


In [141]:
df_1 = df19_1.merge(df18_1, on = ['F18E1'], how= 'outer').fillna(0)
print(df_1.shape)
df_1.head()

(27, 13)


,F19E1,A_19,B_19,C_19,D_19,_19,0_19,F18E1,A_18,B_18,C_18,D_18,_18
0,1,8.0,2,89.0,1.0,0.0,0.0,1,8.0,1,90,1.0,0.0
1,2,97.0,1,2.0,0.0,0.0,0.0,2,93.0,5,2,0.0,0.0
2,3,0.0,79,20.0,1.0,0.0,0.0,4,10.0,3,83,4.0,0.0
3,4,1.0,86,6.0,7.0,0.0,0.0,5,1.0,83,7,9.0,0.0
4,5,5.0,3,87.0,5.0,0.0,0.0,3,1.0,26,71,1.0,0.0


In [143]:
# def selsub(x):
#     if isinstance(x,tuple):
#             if len([val for val in x[0] if np.isnan(val)])>0|len([val for val in x[1] if np.isnan(val)])>0:
#                 res = np.nan
#             else:
#                 res = int(x[0])-int(x[1])   
#     return res
        
def optiondiff(df, options):
    """
    returns: pandas.core.series.Series object
    """
    if not isinstance(df, pd.DataFrame):
        print('df must be pd.DataFrame object not ',type(df))
    if not isinstance(options,list)|isinstance(options,str):
        print('options must be a list or str')
    complist = dict()
    for choice in options:
        tmp = [col for col in df_1.columns if choice in col]
        complist.update({choice:tmp})
#     print(complist['A'][1])#[0] for key in complist.keys())
    for key in complist.keys():
#         compcolumn0 = complist[choice][0]
#         compcolumn1 = complist[choice][1]
        df[key+'_diff'] = df[complist[key][0]]-df[complist[key][1]]
#         print(compcolumn0,compcolumn1)
        

In [94]:
options = ['A','B','C','D',' ','0']
for choice in options:
        tmp = [col for col in df_1.columns if choice in col]
        print(choice+'_diff',tmp)

A_diff ['A_19', 'A_18']
B_diff ['B_19', 'B_18']
C_diff ['C_19', 'C_18']
D_diff ['D_19', 'D_18']
 _diff [' _19', ' _18']
0_diff ['0_19']


In [146]:
optiondiff(df_1,['A','B','C','D',' '])
df_1.to_csv(tu['workoutput']+'/Exam1_18-19comp.csv')
df_1

,F19E1,A_19,B_19,C_19,D_19,_19,0_19,F18E1,A_18,B_18,C_18,D_18,_18,A_diff,B_diff,C_diff,D_diff,_diff
0,1,8.0,2,89.0,1.0,0.0,0.0,1,8.0,1,90,1.0,0.0,0.0,1,-1.0,0.0,0.0
1,2,97.0,1,2.0,0.0,0.0,0.0,2,93.0,5,2,0.0,0.0,4.0,-4,0.0,0.0,0.0
2,3,0.0,79,20.0,1.0,0.0,0.0,4,10.0,3,83,4.0,0.0,-10.0,76,-63.0,-3.0,0.0
3,4,1.0,86,6.0,7.0,0.0,0.0,5,1.0,83,7,9.0,0.0,0.0,3,-1.0,-2.0,0.0
4,5,5.0,3,87.0,5.0,0.0,0.0,3,1.0,26,71,1.0,0.0,4.0,-23,16.0,4.0,0.0
5,6,1.0,53,5.0,41.0,0.0,0.0,6,11.0,7,12,71.0,0.0,-10.0,46,-7.0,-30.0,0.0
6,7,7.0,16,75.0,2.0,0.0,0.0,10,6.0,10,83,1.0,0.0,1.0,6,-8.0,1.0,0.0
7,8,72.0,20,3.0,5.0,1.0,0.0,7,76.0,16,2,4.0,1.0,-4.0,4,1.0,1.0,0.0
8,9,20.0,4,70.0,6.0,0.0,0.0,8,21.0,7,64,8.0,0.0,-1.0,-3,6.0,-2.0,0.0
9,10,14.0,48,35.0,3.0,0.0,0.0,9,10.0,56,29,4.0,0.0,4.0,-8,6.0,-1.0,0.0


In [155]:
abs(df_1[diffcols]).apply(max, axis=1)

0      1.0
1      4.0
2     76.0
3      3.0
4     23.0
5     46.0
6      8.0
7      4.0
8      6.0
9      8.0
10     1.0
11    20.0
12    12.0
13     4.0
14     3.0
15     6.0
16    54.0
17    82.0
18     4.0
19    11.0
20     6.0
21     3.0
22     2.0
23     3.0
24     3.0
25     5.0
26     6.0
dtype: float64

## Identifying Big Changes
Here we identify changes greater than the absolute value of a threshold (arbitrarily set at 10 here).

In [158]:
threshold = 10
diffcols = [col for col in df_1.columns if 'diff' in col]
df_1.loc[abs(df_1[diffcols]).apply(max, axis=1)>threshold,['F19E1','F18E1']+diffcols]

,F19E1,F18E1,A_diff,B_diff,C_diff,D_diff,_diff
2,3,4,-10.0,76,-63.0,-3.0,0.0
4,5,3,4.0,-23,16.0,4.0,0.0
5,6,6,-10.0,46,-7.0,-30.0,0.0
11,12,12,-20.0,14,-1.0,6.0,0.0
12,13,21,10.0,-5,-12.0,8.0,0.0
16,17,17,-31.0,54,8.0,-29.0,-1.0
17,18,18,82.0,-7,8.0,-82.0,0.0
19,20,23,1.0,-11,3.0,6.0,0.0
